<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

In [59]:
import pandas as pd
import folium
from folium import Map, Marker, PolyLine, features, RegularPolygonMarker, DivIcon
from folium.plugins import PolyLineTextPath
import math
import networkx as nx
import time
import concurrent
import multiprocessing

In [ ]:
# 2.3.1

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep = "\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep = "\t")

charging_station_nodes = {323346405, 87120378, 2374444198, 1345740157, 2351742223}

coords = {
    row['osmid']: (row['y'], row['x'])  # Folium notar (lat, lon)
    for _, row in nodes.iterrows()
}

# búum til graf með stefnu og þyngd fyrir edges
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

## Need to reverse in some cases because it's directed
G_rev = G.reverse()

In [60]:
# 2.3.2

def shortest_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.dijkstra_path_length(graph, source=node_id, target=charger_id, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

In [ ]:
# 2.3.3

def process_node_wrapper(args):
    row, charger_nodes, graph, charger_only = args
    node_id = row['osmid']
    if node_id in charger_nodes:
        color = "orange"
        popup_text = f"🔌 Hleðslustöð <br>Node ID: {node_id}"
    elif charger_only == False:
        color = "red" if row['primary'] else "blue"
        distance, closest = shortest_distance_to_charger(node_id, graph, charger_nodes)
        popup_text = (f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>"
                      f"Fjarlægð frá næstu hleðslustöð: {distance:.2f} meters<br>"
                      f"Næsta hleðslustöð: {closest}")
    else:
        return None
    
    return {
        'location': [row['y'], row['x']],
        'color': color,
        'popup_text': popup_text
    }

# keyrsla gæti tekið sirka 5-10 mín
def create_map(charger_nodes, map_name, chargers_only = False):
    center_lat = nodes['y'].mean()
    center_lon = nodes['x'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    start_time = time.time()

    row_args = [
        (row, charger_nodes, G, chargers_only)
        for _, row in nodes.iterrows()
    ]

    # ryzen 7 go brrrr
    with multiprocessing.Pool() as pool:
        marker_data_list = pool.map(process_node_wrapper, row_args)

    for md in marker_data_list:
        if md is None:
            continue
        
        folium.CircleMarker(
            location=md['location'],
            radius=4,
            color=md['color'],
            fill=True,
            fill_color=md['color'],
            fill_opacity=0.8,
            popup=folium.Popup(md['popup_text'], max_width=250)
        ).add_to(m)

    print(f"Keyrsla tók {int(time.time() - start_time)}s")

    for _, row in edges.iterrows():
        if row['u'] in coords and row['v'] in coords:
            start = coords[row['u']]
            end = coords[row['v']]
            PolyLine(
                locations=[start, end],
                color='gray',
                weight=1,
                opacity=0.5,
                popup=row.get('name', '')
            ).add_to(m)

            # Reiknum gráðu á ör-iconinu og setjum svo á endann á línunni
            # vantar að laga betur, gerir kortið mjög hægt líka.
            """angle = calculate_angle(start, end)
            folium.map.Marker(
                location=end,
                icon=DivIcon(
                    icon_size=(150, 36),
                    icon_anchor=(7, 20),
                    html=f'<div style="transform: rotate({angle}deg); color: gray; font-size: 16px;">&#8594;</div>'
                )
            ).add_to(m)"""

    m.save(map_name)
    print(f"Kort geymt i skra: {map_name}")

create_map(charging_station_nodes, "kort_2_3_3.html")

Keyrsla tók 26s
Kort geymt i skra: kort_2_3_3.html


In [62]:
# 2.3.4

## it's in the code above

In [ ]:
# 2.3.5

## Heuristic for A*
## Needlessly complicated on such a small scale
## Calculate the distance between two points on the surface of a sphere
## It's somewhat off up here since the Earth isn't a perfect sphere

def calc_spherical_distance(node_id, target_id):
    lat_source,lon_source = coords[node_id]
    lat_target,lon_target = coords[target_id]

    radius = 6373.0

    delta_lat = math.radians(lat_target) - math.radians(lat_source)
    delta_lon = math.radians(lon_target) - math.radians(lon_source)

    a = math.sin(delta_lat / 2)**2 + math.cos(lat_source) * math.cos(lat_target) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000 #metres

    return distance

def astar_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.astar_path_length(graph, source=node_id, target=charger_id, heuristic=calc_spherical_distance, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

def process_astar_row(args):
    row, charging_station_nodes, graph = args
    node_id = row['osmid']

    distance, closest = astar_distance_to_charger(node_id, graph, charging_station_nodes)
    return distance if distance != float("inf") else 0.0

def run_astar(charging_station_nodes, benchmark):
    distance_sum = 0.0
    if benchmark:
        start_time = time.time()

    row_args = [(row, charging_station_nodes, G) for _, row in nodes.iterrows()]
    
    with multiprocessing.Pool() as pool:
        distances = pool.map(process_astar_row, row_args)

    distance_sum = sum(distances)

    if benchmark:
        print(f"Keyrsla tók {int(time.time() - start_time)}s")

    return distance_sum

run_astar(charging_station_nodes, True)

Keyrsla tók 18s


52496231.653249905

In [64]:
# 2.3.6

## For all primary nodes, run Dijkstra on the reversed graph
def simple_dijkstra(charger_id):
    distances = nx.single_source_dijkstra_path_length(G_rev, charger_id, weight='weight')
    distance_sum = sum(distances.values())

    return distance_sum

def multi_dijkstra(candidate_charger_id: int, placed_charger_ids: set):
    chargers = placed_charger_ids.copy()
    chargers.add(candidate_charger_id)
    
    distances = nx.multi_source_dijkstra_path_length(G_rev, chargers, weight="weight")
    distance_sum = sum(distances.values())

    return distance_sum

def place_optimal_charger(primary_nodes, used_nodes):
    best_distance = float("inf")
    best_node = None

    for node in primary_nodes:
        distance = multi_dijkstra(node.osmid, used_nodes)

        if distance < best_distance and distance != 0:
            best_distance = distance
            best_node = node

    return best_node, best_distance

primary_nodes = []
for _, row in nodes.iterrows():
    if row['primary'] == True:
        primary_nodes.append(row)

optimal_node, distance = place_optimal_charger(primary_nodes, set())
print(f"Best node for k = 1 was {optimal_node} with a distance of {distance}")

Best node for k = 1 was osmid       34827739
x         -21.845736
y          64.114075
primary         True
Name: 589, dtype: object with a distance of 71813532.58901912


In [66]:
# 2.3.7

## Always add the new best node. Obviously have to remove the previously best node

def greedy_k_chargers(max_iterations, used_nodes):
    best_nodes = []
    total_distance = 0.0

    # If we already used some nodes, don't include them here
    for n in used_nodes:
        primary_nodes[:] = [node for node in primary_nodes if not node.equals(n)]

    start_time = time.time()
    for k in range(max_iterations):
        best_node, dist = place_optimal_charger(primary_nodes, used_nodes)
        total_distance += dist
        best_nodes.append(best_node)
        used_nodes.add(best_node.osmid)
        # Python is awful btw
        primary_nodes[:] = [node for node in primary_nodes if not node.equals(best_node)]

    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return best_nodes, total_distance

greedy_nodes, greedy_dist = greedy_k_chargers(10, set())
print(f"Cumulative distance was {greedy_dist}")
create_map([node.osmid for node in greedy_nodes], "kort_2_3_7.html", True)

Keyrsla tók 202s
Cumulative distance was 384125892.5509423
Keyrsla tók 1s
Kort geymt i skra: kort_2_3_7.html


In [67]:
# 2.3.8

## Pick random starting node
### Run greedy K for 5 nodes
#### Pick best node, then choose the furthest of the remaining 4
##### Recurse
###### Pick solution with smallest sum of distances at the end

def better_greedy_k(max_depth):
    # Choose a random starting node
    starting_node = nodes.sample(n=1).iloc[0]['osmid']
    
    start_time = time.time()
    chain, total_distance = recursive_search(starting_node, {int(starting_node)}, max_depth)
    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return chain, total_distance

# helper outside to do recursive things in parallel
def recurse(candidate, new_used, depth_left):
    return recursive_search(candidate, new_used, depth_left)

def recursive_search(current_node: int, used_nodes: set, depth):
    if depth == 0:
        # Done
        return [current_node], 0

    best_candidate, second_candidate = pick_2(used_nodes)
    
    cost_best_edge = multi_dijkstra(best_candidate.osmid, used_nodes)
    cost_second_edge = multi_dijkstra(second_candidate.osmid, used_nodes)

    new_used_best = used_nodes.copy()
    new_used_best.add(best_candidate.osmid)

    new_used_second = used_nodes.copy()
    new_used_second.add(second_candidate.osmid)

    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
        # Submit both branches
        future_best = executor.submit(recurse, best_candidate.osmid, new_used_best, depth - 1)
        future_second = executor.submit(recurse, second_candidate.osmid, new_used_second, depth - 1)

        # Wait for both to complete
        chain_best, _ = future_best.result()
        chain_second, _ = future_second.result()

    # Choose the branch with lower cost
    if cost_best_edge < cost_second_edge:
        return [int(current_node)] + chain_best, cost_best_edge
    else:
        return [int(current_node)] + chain_second, cost_second_edge
    
def pick_2(used_nodes):
    # Run greedy for 5 nodes
    chargers, distances = greedy_k_chargers(5, used_nodes)
    # First charger
    best_charger = chargers.pop(0)
    second_best = None

    max_distance = 0.0

    # Pick charger futhest from the best
    for k in chargers:
        dist = calc_spherical_distance(k.osmid, best_charger.osmid)
        if dist > max_distance:
            max_distance = dist
            second_best = k

    return best_charger, second_best

better_greedy_nodes, better_greedy_dist = better_greedy_k(10)
print(f"Cumulative distance was {better_greedy_dist}")
create_map(better_greedy_nodes, "kort_2_3_8.html", True)

Keyrsla tók 98s
Keyrsla tók 104s
Keyrsla tók 105s
Keyrsla tók 118s
Keyrsla tók 119s
Keyrsla tók 119s
Keyrsla tók 119s
Keyrsla tók 147s
Keyrsla tók 147s
Keyrsla tók 147s
Keyrsla tók 147s
Keyrsla tók 148s
Keyrsla tók 148s
Keyrsla tók 148s
Keyrsla tók 149s
Keyrsla tók 215s
Keyrsla tók 216s
Keyrsla tók 216s
Keyrsla tók 218s
Keyrsla tók 218s
Keyrsla tók 220s
Keyrsla tók 217s
Keyrsla tók 220s
Keyrsla tók 217s
Keyrsla tók 219s
Keyrsla tók 219s
Keyrsla tók 230s
Keyrsla tók 229s
Keyrsla tók 230s
Keyrsla tók 232s
Keyrsla tók 239s
Keyrsla tók 426s
Keyrsla tók 430s
Keyrsla tók 436s
Keyrsla tók 442s
Keyrsla tók 435s
Keyrsla tók 452s
Keyrsla tók 441s
Keyrsla tók 434s
Keyrsla tók 456s
Keyrsla tók 447s
Keyrsla tók 460s
Keyrsla tók 464s
Keyrsla tók 459s
Keyrsla tók 469s
Keyrsla tók 473s
Keyrsla tók 456s
Keyrsla tók 462s
Keyrsla tók 481s
Keyrsla tók 480s
Keyrsla tók 491s
Keyrsla tók 499s
Keyrsla tók 490s
Keyrsla tók 509s
Keyrsla tók 507s
Keyrsla tók 497s
Keyrsla tók 500s
Keyrsla tók 498s
Keyrsla tók 527